[Schema Networks: Zero-shot Transfer with a Generative Causal Model of Intuitive Physics](https://arxiv.org/pdf/1706.04317.pdf)

### Препрцессинг данных:
![ ](img1.png)

### Схемы:
Для момента $t$ набор схем W предсказывает j-ые атрибуты или награду в мемент $t+1$ :

$$ y =  \overline{\overline{X} W}\vec{1} = f_W(X)$$

где сложение - логическое или, черта -  отрицание. Значение W - решение оптимизационной задачи:
$$ min_{W} \frac{1}{RM}|y - f_W(X)|_1 + C|W|_1$$
Алгоритм приближенного решения предлагается в конце статьи.
![ ](img4.png)
Схемы задают граф переходов между состояниями, где все возможные переходы равновероятны:
![ ](ing3.png)


### Планирование действий:
Авторы используют max-product belief propagation (MPBP), который состоит из стадий:
* **Анализ возможных достижений:**
* **Выбор цели с положительной наградой:**
* **Избегание состояний с отрицательной наградой:**
* **Backtracking:**

Алгоритм описан в [Planning by Probabilistic Inference](https://pdfs.semanticscholar.org/01aa/f5722c0bcb06d536a359e4a2223c7755e8f3.pdf?_ga=2.89309403.1549993353.1563450871-1418804146.1563450871) (вроде он, но в статье он так не называется)

### Общий алгоритм:
1. **Преобразование входного изображения:** получить набор сущностей с бинарными атрибутами, используя, например, [autoencoders](https://arxiv.org/pdf/1609.05518.pdf).
2. **Преобразование данных:** построить матрицу $X^t$: где i-ый столбец состоит из атрибутов i-ой сущности и атрибутов ее соседей для заданного радиуса в момент $t$
3. **Обучение:** Пусть на предыдущем шаге было совершено действие а, тогда:
    *  $T = \{t:$ в момент $t$ было совершено $a\}$   <font color='orange'>(о том, что для каждого действия своя схема упоминается только в секции 3.2, но это в целом логично)</font>
    *  $X = \begin{pmatrix} X_{t_1} \\ ... \\ X_{t_{|T|}} \end{pmatrix}, t_i \in T$
    *  для каждого атрибута $j$: $y =  \begin{pmatrix} \alpha_{1j}^{t_1 + 1} \\ ... \\ \alpha_{Mj}^{t_{|T|}+1} \end{pmatrix}$ - атрибуты после действия  $a$ <font color='orange'>(я так поняла, что аналогично предсказываются награды)</font>
    *  поиск оптимальных схем: $W = argmin_{W} \frac{1}{RM}|y - f_W(X)|_1 + C|W|_1$
        *  $x_n| f_W(x_n)=0$
        * добавим один $x_n$ в множество $S$
        * добавление схемы:
         $$min_w \sum  \limits_{n:y_n =1}(1−x_n)w$$
         $$(1 − x_n)w >1|y_n=0$$
         $$(1 − x_n)w = 0| x_n \in S$$
        * добавим все $x_n$ $| (1-x_n)w = 0 $ в $S$
        * упрощение схемы:
         $$min_w|w|_1$$
         $$(1 − x_n)w >1|y_n=0$$
         $$(1 − x_n)w = 0| x_n \in S$$
        * если полученная схема не бинарна, то повторить пердыдущий шаг для ненулевых элеменотов схемы, используя бинарную оптимизацию 
4. **Выбор дейстовия [Planning by Probabilistic Inference]:**
    * $s_1$ - текущее состояние, $T$ - временное окно 
    * Поиск достижимого состояния с положительной наградой (Forward Pass):
        * для всех $ n = 2, ..., T$: 
         $$q_n(s′,a′)= \sum\limits_{a,s}q_n(s′,a′ |s,a)q_{n−1}(s,a)$$
         $$q_n(a′ | a) = \frac{\sum\limits_{s, s'}q_n(s', a'|s ,a)q_{n-1}(s, a)}{ \sum_sq_{n−1}(s,a)}$$
        * среди всех положительных  наград выберем ту, которая достигаеся раньше и присвоим ей 1
        * присвоим 0 тем отрицательным наргадам, для которых такая позиция достижима 
    * Поиск последовательности действий, ведущей к желаемому состоянию:
        * Комбинация Viterby и поиска в глубину <font color='orange'>(Тут не совсем понятно: в данном алгоритме нет вероятности, как таковой (действие и состояние однозначно задают следующее состояние), а все условные вероятности просто индикаторы. При этом отрицательной награды хочется избегать на всем пути следования. Так что витерби, может, стоит применять как раз для минимизации наказания? Эта часть алгоритма вообще очень расплывчато описана.)</font>